# Dataset
- PolymerDB

# Scoring
- Cross-validatoin

# Descriptors
- All
- Selected by Boruta
- Selected by GPT-4

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


In [2]:
path="database/forpub2.csv"
original_df=pd.read_csv(path,index_col=0)
dataset_df=original_df
dataset_df

,tags,title,reference,SMILES,Experimental n,DFT_energy,DFT_dipoleX,DFT_dipoleY,DFT_dipoleZ,DFT_dipoleTot,...,JR_EnthalpyVap,JR_EnthalpyFus,JR_LiquidVisco,JR_CrystalSolub_1,JR_CrystalSolub_2,JR_AmorphSolub_1,JR_AmorphSolub_2,DFTMD_vol,Experimental_volume,Experimental_alpha
id,,,,,,,,,,,,,,,,,,,,,
9259,4,"Poly(propylene), Poly(1-methylethylene)",PolymerDatabase https://polymerdatabase.com/,CC([*])C[*],1.4850,-119.155360,0.0000,0.0493,0.0000,0.0493,...,21.590,3.367,0.000109,0.000000,0.000000,20020.439760,10033.988820,118.891063,80.811553,37.312942
9338,4,"Poly(isobutene), Poly(isobutylene), Butyl rubber",PolymerDatabase https://polymerdatabase.com/,[*]C(C)(C)C[*],1.5075,-158.472765,0.0000,0.0000,-0.0768,0.0768,...,22.908,2.066,0.000216,0.000000,67410.071580,44772.610010,22439.460550,148.976413,106.517323,51.111260
9279,4,"Poly(1,2-butadiene)",PolymerDatabase https://polymerdatabase.com/,[*]CC(C=C)[*],1.5400,-157.232796,-0.3333,0.0169,-0.0949,0.3469,...,23.146,4.677,0.000145,0.000000,0.000000,8887.086993,4454.094545,143.262170,98.740462,49.910206
9304,4,"Poly(ethyl ethylene), Poly(butylene)",PolymerDatabase https://polymerdatabase.com/,CCC([*])C[*],1.5000,-158.471954,0.0000,0.0000,0.0000,0.0000,...,23.816,5.957,0.000163,0.000000,0.000000,2292.648030,1149.045924,148.506056,108.501348,51.411865
9318,4,Poly(octylethylene),PolymerDatabase https://polymerdatabase.com/,CCCCCCCCC([*])C[*],1.4700,-394.371020,0.0000,0.0000,0.0000,0.0000,...,37.172,21.497,0.000824,483.130880,242.139029,7.644375,3.831263,368.858948,269.216227,121.008510
9340,4,Poly(isobutylethylene),PolymerDatabase https://polymerdatabase.com/,CC(C)CC([*])C[*],1.4650,-237.104290,0.0146,0.0311,-0.0695,0.0776,...,27.880,7.614,0.000307,58548.536990,29343.779290,426.397540,213.705003,218.652426,167.429924,74.568814
9341,4,"Polyisoprene, Poly(1-methyl-1-butenylene)",PolymerDatabase https://polymerdatabase.com/,[*]C/C=C(C[*])\C,1.5150,-196.556298,0.0851,0.2494,0.0000,0.2635,...,26.468,10.962,0.000000,0.000000,70594.076920,112.768146,56.517955,167.521892,125.032580,60.742077
9284,4,Poly[oxy(hexyloxymethyl)ethylene],PolymerDatabase https://polymerdatabase.com/,CCCCCCOCC([*])OC[*],1.4700,-505.457096,0.1800,-0.1464,0.0000,0.2320,...,39.766,21.283,0.000867,29162.115030,14615.679760,4929.827086,2470.766400,382.419404,270.988458,121.805100
9264,4,"Polystyrene, Polyethenylbenzene, Lustrexﾂｮ, St...",PolymerDatabase https://polymerdatabase.com/,[*]CC([*])c1ccccc1,1.5250,-310.895081,-0.2966,0.0000,-0.0073,0.2967,...,34.996,10.358,0.000608,20595.275890,10322.089350,3293.740889,1650.780884,249.156258,164.771397,81.350605


In [3]:
from reg_utils.regression import AutoBoruta
autoboruta=AutoBoruta()

In [4]:
col_list=list(dataset_df.drop(columns=['tags','title','reference','SMILES','Experimental n']).columns)

import random
random.seed(0)
r_col_list10=random.sample(col_list,10)
r_col_list20=random.sample(col_list,20)

In [5]:
from sklearn.linear_model import RidgeCV,LassoCV
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

dataset_dict={
    "Dataset":dataset_df,
}
cv_dict={
"Random CV":True,
}
descriptor_dict={
    "All":[],
    "Random-10":r_col_list10,
    "Random-20":r_col_list20,
    "Boruta":[],
    "GPT-4":["DFT_energy", "DFT_HOMO", "DFT_LUMO", "DFT_dipoleTot", "rdkit_MolWt", "rdkit_NumValenceElectrons", "rdkit_NumAromaticRings", "rdkit_NumAliphaticRings", "DFTMD_vol", "rdkit_HeavyAtomMolWt", "rdkit_MaxAbsPartialCharge", "rdkit_MolLogP", "JR_BoilingPoint", "JR_EnthalpyForm"],
}
model_dict={
    "Ridge":RidgeCV(),
    "Lasso":LassoCV(),
    "SVR":SVR(),
    "GPR":GaussianProcessRegressor(kernel=RBF()+WhiteKernel()),
    "RFR":RandomForestRegressor(n_jobs=-1),
    "GBR":GradientBoostingRegressor(),
}

In [24]:
import joblib
import datetime
res_dict={}
k=5
for dataset_name,dataset in dataset_dict.items():
    for cv_name,cv in cv_dict.items():
        for descriptor_name,descriptor in (descriptor_dict.items()):
            for model_name,model in model_dict.items():
                if descriptor_name=="Boruta":
                    do_boruta=True
                else:
                    do_boruta=False
                res_dict[(dataset_name,cv_name,descriptor_name,model_name)]=auto_evaluation(dataset, model, 
                                                                                            random_CV=cv,
                                                                                              selected_descriptors=descriptor,
                                                                                              autoboruta=autoboruta,
                                                                                              do_boruta=do_boruta,
                                                                                              k=k)
                print((dataset_name,cv_name,descriptor_name,model_name))

today=datetime.datetime.now()
today_str=today.strftime('%Y%m%d%H%M')
joblib.dump(res_dict,f"data/{today_str}.pkl")

('Dataset', 'Random CV', 'All', 'Ridge')
('Dataset', 'Random CV', 'All', 'Lasso')
('Dataset', 'Random CV', 'All', 'SVR')
('Dataset', 'Random CV', 'All', 'GPR')
('Dataset', 'Random CV', 'All', 'RFR')
('Dataset', 'Random CV', 'All', 'GBR')
('Dataset', 'Random CV', 'Random-10', 'Ridge')
('Dataset', 'Random CV', 'Random-10', 'Lasso')
('Dataset', 'Random CV', 'Random-10', 'SVR')
('Dataset', 'Random CV', 'Random-10', 'GPR')
('Dataset', 'Random CV', 'Random-10', 'RFR')
('Dataset', 'Random CV', 'Random-10', 'GBR')
('Dataset', 'Random CV', 'Random-20', 'Ridge')
('Dataset', 'Random CV', 'Random-20', 'Lasso')
('Dataset', 'Random CV', 'Random-20', 'SVR')
('Dataset', 'Random CV', 'Random-20', 'GPR')
('Dataset', 'Random CV', 'Random-20', 'RFR')
('Dataset', 'Random CV', 'Random-20', 'GBR')
Boruta selected features:  (9259, 9338, 9279, 9318, 9340, 9341, 9284, 9264, 9298, 9241, 9322, 9251, 9242, 9303, 9342, 9316, 9337, 9256, 9262, 9297, 9324, 9320, 9246, 9244, 9243, 9258, 9332, 9310, 9323, 9311, 9281, 

['data/202305201150.pkl']

# Descriptor selection by Lasso, Boruta, etc

In [6]:
from reg_utils.regression import prepare_X_and_y    
from reg_utils.regression import prepare_pipeline

X, y = prepare_X_and_y(dataset_df)
X=X[descriptor_dict["GPT-4"]]

model=prepare_pipeline(LassoCV())
model.fit(X,y)
coeff=model[-1].coef_

lasso_features=[col for col,coef in zip(X.columns,coeff) if coef!=0]
lasso_features

['DFT_energy',
 'DFT_HOMO',
 'DFT_LUMO',
 'DFT_dipoleTot',
 'rdkit_MolWt',
 'rdkit_NumValenceElectrons',
 'rdkit_NumAromaticRings',
 'rdkit_NumAliphaticRings',
 'DFTMD_vol',
 'rdkit_HeavyAtomMolWt',
 'rdkit_MolLogP',
 'JR_BoilingPoint',
 'JR_EnthalpyForm']

In [8]:
X, y = prepare_X_and_y(dataset_df)
X=X

model=prepare_pipeline(LassoCV())
model.fit(X,y)
coeff=model[-1].coef_

lasso_features=[col for col,coef in zip(X.columns,coeff) if coef!=0]
lasso_features

['DFT_dipoleX',
 'DFT_dipoleY',
 'DFT_LUMO',
 'rdkit_qed',
 'rdkit_BalabanJ',
 'rdkit_SMR_VSA4',
 'rdkit_SMR_VSA7',
 'rdkit_SlogP_VSA12',
 'rdkit_SlogP_VSA8',
 'rdkit_EState_VSA7',
 'rdkit_EState_VSA8',
 'rdkit_VSA_EState3',
 'rdkit_FractionCSP3',
 'rdkit_fr_Al_COO',
 'rdkit_fr_COO',
 'rdkit_fr_COO2',
 'rdkit_fr_NH2',
 'rdkit_fr_priamide',
 'JR_CriticalTemp',
 'JR_GibbsEnergy']